In [ ]:
from fastai.vision.all import *
import sys
import numpy as np
np.set_printoptions(threshold=np.inf)

In [ ]:
import pandas as pd
import os
import random
df = pd.read_csv('../input/rsna-miccai-brain-tumor-radiogenomic-classification/train_labels.csv', header=0, names=['id','true_value'], dtype=object)


In [ ]:
df.head()

In [ ]:
df_test = pd.DataFrame(columns=['id', 'value'])
df_test.id = os.listdir("../input/rsna-miccai-brain-tumor-radiogenomic-classification/test/")

In [ ]:
#https://stackoverflow.com/a/4836734/8245487
def natural_sort(l): 
    convert = lambda text: int(text) if text.isdigit() else text.lower()
    alphanum_key = lambda key: [convert(c) for c in re.split('([0-9]+)', key)]
    return sorted(l, key=alphanum_key)

In [ ]:
import os
import pydicom
import pandas as pd
from pydicom.pixel_data_handlers.util import apply_voi_lut
from tqdm import tqdm
import binascii
from PIL import Image
from multiprocessing import Pool


INPUT = '../input/rsna-miccai-brain-tumor-radiogenomic-classification'

def get_dicom_files(dataset, df):
    for cur_id in df.id:
        pred = get_dicom_files_helper(dataset, cur_id)
        df.loc[df.id==cur_id,'value'] = pred

def get_dark_cells(data):
    return np.sum((data > 60) & (data <= 230), axis=(0,1))

def get_very_bright_cells(data):
    return np.sum((data > 230), axis=(0,1))

def get_live_cells(data):
    return np.sum((data > 60), axis=(0,1))


def get_dicom_files_helper(dataset, cur_id):
    root = '../input/rsna-miccai-brain-tumor-radiogenomic-classification/{}'.format(dataset)
    for scan_type in ['FLAIR', 'T1w', 'T1wCE', 'T2w']:
        cur_dir = os.path.join(root, cur_id, scan_type)
        dicoms = os.listdir(cur_dir)
        if len(dicoms) == 0:
            print('Skipping {}'.format(cur_dir))
            continue
            
        dicoms = natural_sort(dicoms)
        clip = len(dicoms) // 3
        upper_clip = len(dicoms)-clip
        dicoms = dicoms[clip:upper_clip] #take middle 1/3
        num_dicoms = len(dicoms)

        possible_meth = 0 #per image type

        for dicom in dicoms:
            filepath = os.path.join(cur_dir, dicom)
            data = process_dicom(filepath)
            #print(data)
            live_cells = get_live_cells(data)
            if live_cells < 10:
                continue
            dark_cells = get_dark_cells(data)
            bright_cells = live_cells - dark_cells
            very_bright_cells = get_very_bright_cells(data)
            not_very_bright_cells = live_cells - very_bright_cells 
            mostly_dark = (dark_cells/live_cells) > 0.5
            if mostly_dark:
                if (very_bright_cells/live_cells) > 0.20:
                    possible_meth += 1
            #mostly bright picture
            elif (not_very_bright_cells/live_cells) > 0.20: 
                possible_meth += 1
            
            if possible_meth >= (num_dicoms * 0.05):
                print('Its possible!')
                return 1
    
    print('Nope!')
    return 0
        

def process_dicom(path):
    dicom = pydicom.read_file(path)
    data = apply_voi_lut(dicom.pixel_array, dicom)
    if dicom.PhotometricInterpretation == "MONOCHROME1":
        data = np.amax(data) - data
    data = data - np.min(data)
    data = data / np.max(data)
    data = (data * 255).astype(np.uint8)
    
    return data


get_dicom_files('train', df)
get_dicom_files('test', df_test)


In [ ]:
df.value.value_counts()

In [ ]:
df.true_value.value_counts()

In [ ]:
df[df.value==0]

In [ ]:
df_test.value.value_counts()

In [ ]:
df_test.rename(columns={'id':'BraTS21ID','value':'MGMT_value'}).to_csv('submission.csv', index=False)